CrystalineFileLecturePredict.ipynb

Reads from D3Files all the files it is going to process

Outputs the following folders and files:

1. CrystallineLog_Predicting_Creation.txt
Logs all the prints and every step the code does. If you trust the code, it is irrelevant. If you don't trust it or want to change it then this txt file will tell you how each experiment file has been processed and where there might have been issues.


2. Crystalline_CellID
Contains the Cell IDs found on all the files. Needed for the ML code.


3. CrystallineSeparatedFolder
It will create a folder for each subfolder where .fli files that could be used were found. Only the numerical giles are saved, a.k.a, {base_name}.txt. They contain DeltaTime (the time of the measurement measured from the first VALID polarization measurement), PolarizationD3, SoftPolarizationD3 (the polarization after using a Savitzky-Golay filter) and ErrPolarizationD3 (the uncertainty). The parameter files are directly sent to the next folder. Despite having duplicates of the information, this folder has been saved as it also has any intermediate file that has not been fully processed. If one of the .fli files has any issues, the file is saved as is before running to the issue. 

4. ML/CrystallinePredictFiles
This folder is not inside the folder you are currently in (FileReadingStoring) but on another one at the same level called ML. It contains the files nneded for the ML predictions.

4.1 {base_name}.txt
Contain DeltaTime (the time of the measurement measured from the first VALID polarization measurement), PolarizationD3, SoftPolarizationD3 (the polarization after using a Savitzky-Golay filter) and ErrPolarizationD3 (the uncertainty).

4.2 {base_name}_Parameters.txt
Contains the CellID, Pressure, LabPolarization (the polarization measured at the lab) and LabTimeCellID (the time when it was measured)


5. CrystallineDataBase
Contains all the .fli files that were attempted to be read before manipulating them


6. CrystallineBadFiles
Contains all the .fli separated in experiment sets folders that were rejected (not enough points, negative polarizations, etc.)

7. PolarizationTImeReference
The models where trained using relative time only. This means that the first valid polarization measurement has been considered as time zero for each experiment and the rest of measurements have their associated time values as a variation of time since that reference. This way all experiments have the same structure. However this reference time is not absolute and the measurements of the diffractograms may have an absolute time reference different from the ones used in the models. By saving the string "Year-Month-Day Hour:Minute:Second" we can safely change the time reference


_________________________________________________________________________________________

Process it follows:

1. REMOVAL OF PREVIOUS ITERATIONS
To avoid leaks and duplications, all files are erased before running the code file

2. ZIP FOLDER TREATMENT
The code will take all the zip files, extract them, remove duplicates using the name AND hash sha256.

3. SEPARATION OF FLI FILES ACCORDING TO EXPERIMENTS

Some fli files have the wrong structure (they are not polarization measurementes) and if they are polarization files they can have more than one experiment per file.
For evey fli file we will read the contents and try to find the header (a string in an entire line). This symbolizes the beginning of an experiment
If there are numerical values before the first header, that means that the process of saving the file occured before changing something of the experiment. These data rows will be skipped
A correct fli file will have the following structure:
    polariser cell info ge18004 pressure/init. polar 2.29 0.79 initial date/time 17 09 23 @ 10:39
    37391   4.000   0.000   1.000 18/09/23 06:20:44     155.03  +z +z     0.8391    0.0156   11.4270    1.2031     120.00
    37392   4.000   0.000   1.000 18/09/23 06:26:49     155.05  +x +x     0.8255    0.0110   10.4610    0.7211     300.00
    ...

Which corresponds to the following information:
    String:'polariser cell info', CellID, String:'pressure/init. polar', Pressure(unknown units), InitialLabPolarization, String:'date/time', Day, Month, Year, String:'@', Hour:Minute
    Measurement Number, First Miller Index, Second Miller Index, Third Miller Index, Date Of Measurement, Time Of Measurement, Temperature [Kelvin],
                        Direction Of Polarization In The First Polarizer Cell (Direction of the quantum operator S_x,S_y,S_z), Direction Of Polarization In The Second Polarizer Cell,
                        Polarization, Polarization uncertainty, Flipping Ratio, FlippingRatio Uncertainty, Duration of the measurement

The direction +z is chosen to be pointing away from the ground.
The direction +x is the direction of the flow of neutrons, i.e, the direction of Scattering.
The direction +y is the orthogonal to both of them.
D3 uses two polariser cells, one between the reactor and the sample and a second between the sample and the sensor. The first one guarantees that only neutrons with the correct spin direction
interacts with the sample. The second one guarantees that only the neutrons that have unchanged spin direction after interacting with the sample are detected by the sensor. This is
the reason why the directions (+z,+y,+x,-z,-y,-x) appear twice.
We have considered that temperature is not a relevant factor and the flipping ratio has no new information that polarization alrady posseses.
First, the code will first locate the first header (ignoring eveything before) and save all the data afterwards (until the next header or end of the document) in a file with the suffix Array_{i} (i is the number of headers already processed in that fli file)
Second, it will save the header as a file with the suffix Parameters.
Third, the header row and the columns of Measurement Number, Temperature, Flipping Ratio, FlippingRatio Uncertainty and Time Between Measurements will be erased
Fourth, as all data measurement uses the +z,+z combination, all other combinations are erased
Fifth, not all data from all Miller Index combinations are polarization measurements. Even some of the ones that are polarization measurements are tampered (playing with magnetic fields for example).
This means that there needs to be a way to select the correct combination. For starters, irrational Miller indices are not used for measurements with the samples (they need to be discarded)
The integer Miller indices combination will be put to the test by all the functions defined before.
Sixth, It computes a score depending on how many derivatives are negative, (200 / (200 - percent_neg) - 1) to be precise. This is a normalized score (0-1) with a 1/x evolution. Also it computes a score depending of the size of N, 2 * (-0.5 + 1 / (1 + needed_N / 8.54e-2)) to be precise. 8.54e-2 is the maximum of the data set. If a new maximum is achieved, the score wont be normalized (0-1) but won't break. The final score combines both of these values (addition). Manually I have seen that 1.4 is a good threshold. If Score>1.4 the file will be accepted. If it is smaller it will be discarded by the main code (a False will be returned). It does the m<0 test, writes everything in Summary_txt (filename, Score associated with N, Score associated with Derivatives nad the total Score). If the file was chosen to be forcefully accepted or rejected, a string will appear in the .txt file. Finally it will save plots of both filters in PlotResults if it is correct and in FailureTest if it is considered a bad file. Again, if a new file is added it may be wise to check your experiment in these folders. For more info read the description of FilteringMethodInt = 0 inside the code
Seventh, it will try each Miller index combination for a set i value, apply a filter, and return filtered df + PrettyCombination. it will only add the filtered column if enough points & data changes significantly. If it doesn't change too much, the column PolarizationD3 will be duplicated with the new name
Eight, it repeats the process of obtaining the area in m*x+n-N < y < m*x+n+N where 75% of the points are inside the area. It also multiplies the value of N by a factor AcceptableMultiplier and erases all points outisde this bigger area. As uncertainties are clearly underestimated I tried to make them reasonable (looking at the dispersion of the points it is clear there is systematic uncertainties. Under the hypothesis that the polarization curve should be a soft curve (at least C^1) we will try to use χ^2 to add a provisional uncertainty margin fitting to a linear expression. This is a very inaccurate uncertainty increase but it is an improvement of the underestimated uncertainties (and the lack of ways to quantify the systematic uncertainty sources). The enlarged area will be plotted and saved in PlotResults for both the normal data and the softened data (Savitzky–Golay filter)    
    
4. CellID SAVING
It will safely store in a txt file all the cell ids so that the code in ML can use them

5. DUPLICATION REMOVAL
It will check if the files created for the ML algorithm are duplicates and erases them in that case


________________________________________________________________________________________

VERY VERY IMPORTANT

The other code files should not have an issue with the structure inside the fli files as long as you download them from the online data base and keep them zipped. However here we expect a very specific structure. PLEASE HAVE THE FILES INSIDE THE ZIP FILES HAVE THE FOLLOWING STRUCTURE (perhaps in the future we can change it so it is also automatic).
1. Only two rows of numerical values per 'polariser cell info' row
2. The same miller indices for those two rows
3. Direction +z,+z
Here you have an example:

polariser cell info ge18004 pressure/init. polar 2.30 0.78 initial date/time 06 06 25 @ 15:15
   59685   0.000   0.000   2.000 06/06/25 16:20:21     228.88  +z +z     0.8460    0.0007   11.9879    0.0558     180.00
   60273   0.000   0.000   2.000 10/06/25 15:36:07     170.95  +z +z     0.5454    0.0023    3.3998    0.0215      60.00
polariser cell info ge18012 pressure/init. polar 2.30 0.78 initial date/time 10 06 25 @ 15:45
   60275   0.000   0.000   2.000 10/06/25 15:47:23     170.14  +z +z     0.8363    0.0011   11.2157    0.0764      60.00
   60343   0.000   0.000   2.000 11/06/25 14:30:33     169.83  +z +z     0.8016    0.0021    9.0804    0.1047      60.00
polariser cell info ge18004 pressure/init. polar 2.30 0.78 initial date/time 11 06 25 @ 14:30
   60345   2.000   0.000   0.000 11/06/25 14:33:57     169.83  +z +z     0.8019    0.0021    9.0935    0.1039      60.00
   60565   2.000   0.000   0.000 13/06/25 09:43:00     179.66  +z +z     0.6776    0.0033    5.2033    0.0629      60.00
polariser cell info ge18012 pressure/init. polar 2.30 0.78 initial date/time 13 06 25 @ 09:44
   60567   2.000   0.000   0.000 13/06/25 09:49:03     179.66  +z +z     0.8345    0.0018   11.0877    0.1322      60.00
   60886   2.000   0.000   0.000 15/06/25 07:23:31      75.02  +z +z     0.5979    0.0036    3.9735    0.0443      60.00
polariser cell info ge18004 pressure/init. polar 2.30 0.79 initial date/time 15 06 25 @ 10:40
   60888   2.000   0.000   0.000 15/06/25 10:49:27      75.02  +z +z     0.8467    0.0019   12.0495    0.1627      60.00
   60908   2.000   0.000   0.000 16/06/25 05:42:08      75.02  +z +z     0.8059    0.0024    9.3026    0.1256      60.00



In [ ]:
%reset -f

"""
1- LIBRARIES
""" 
import os
import shutil
import zipfile
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import pandas as pd
from datetime import datetime
from pathlib import Path
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit
from collections import defaultdict
from typing import Optional
import hashlib
import zipfile


""" 
2- PRINTING AND LOG DETAILS. LONG PATH CORRECTIONS
"""
"""
Here we have a custom function for log_messageing and logging everything on a .txt file to be able to know what has happened on the code

"""
PrintDebug = True #This Bool will determine if all logs should be log_messageed on screen on the Python Notebook. The log writing is always on. If False the code will be faster.
ShowPlot = False #This Bool works the same but with showing on screen the plots (they are always saved even with this variable being False). Reduces program cost if False
log_file_path = os.path.join(".", "CrystallineLog_Reading_Creation.txt")
# Initialize log file at the start of the script
with open(log_file_path, 'w', encoding='utf-8') as log_file:
    log_file.write("=== Log started ===\n")

def log_message(message):
    if PrintDebug:
        print(message)
    with open(log_file_path, 'a', encoding='utf-8') as log_file:
        log_file.write(str(message) + "\n")


def win_long_path(path):
    # Convert to Path and resolve to absolute
    path = Path(path).resolve()

    # Convert to string
    path_str = str(path)

    # Prepend \\?\ if not already present
    if not path_str.startswith("\\\\?\\"):
        path_str = "\\\\?\\" + path_str

    return path_str

"""
3- FUNCTIONS
"""
#3.1-Time function for the conversion of time into seconds
def Time(Day_Ref, Hour_Ref):
    """
    Expected variables (strings) should be in the form
     Day_Ref = 'DD/MM/YY' a.k.a Day/Month/Year
     Hour_Ref = 'HH:MM' a.k.a Hour:Month
    Seconds will be ignored if fed to the function (there is a check before the call of the function in the main code that erases the seconds).
    The function takes this strings and converts them to seconds
    Could be an improvement to consider seconds but the headers don't use seconds, there is no information to prove that the time variables are precise to the second and ~30 seconds is negligible when working with time periods of up to 70000 seconds
    """
    match = re.match(r"(\d+)/(\d+)/(\d+)", Day_Ref)
    if match:
        DD = int(match.group(1))
        MM = int(match.group(2))
        YY = int(match.group(3))
    else:
        raise ValueError(f"Invalid date format: {Day_Ref}")

    # Parse time
    match = re.match(r"(\d+):(\d+):(\d+)", Hour_Ref)
    if match:
        Hour = int(match.group(1))
        Minute = int(match.group(2))
        Second = int(match.group(3))
    else:
        # Try HH:MM
        match = re.match(r"(\d+):(\d+)", Hour_Ref)
        if match:
            Hour = int(match.group(1))
            Minute = int(match.group(2))
            Second = 0
        else:
            raise ValueError(f"Invalid time format: {Hour_Ref}")

    return datetime(YY + 2000 if YY < 100 else YY, MM, DD, Hour, Minute, Second)

#3.2- Time function to compute the difference in time between two sets fo time
def deltatime(AIni,BIni, AFin,BFin):
    """
    There is nothing as "Absolute time". We want to compare durations or intervals of time.
    The A variables are of the type 'DD/MM/YY' and the B ones are 'HH:MM'
     A is for the time moment considered as reference
     B is for the time moment that has been used for measuring something
    """
    time1 = Time(AIni, BIni)
    time2 = Time(AFin, BFin)
    return( int((time2 - time1).total_seconds()))

#3.3 Conversion to integers
def format_combination(comb):
    """
    The Miller Indeces in the are writen as strings with float like numbers in them, for example (4.0,1.0,2.0)
    As only integer Miller Indices are required we can convert them to integers with this function.
    """
    if comb is None:
        return "(None)"
    ints = tuple(int(float(x)) for x in comb)
    return f"({','.join(map(str, ints))})"

#3.4 Folder name changes
def sanitize(name):
    """
    Remove invalid characters for folder names. [<>:"/\\|?*] all turn to _
    """
    return re.sub(r'[<>:"/\\|?*]', '_', name)

#3.5 Savitzky–Golay filter window parameters
def savgol_params_func(n_points):
    """
    Receives tha number of points of the file that the filter will be tried to be used on
    Window length can't be greater than the number of points. To avoid Python Errors this function gives the appropiate window length and order of the polygone
    """
    window_length = min(default_window_length, n_points)
    if window_length % 2 == 0:
        window_length -= 1
    if window_length < polyorder + 2:
        window_length = polyorder + 2
        if window_length % 2 == 0:
            window_length += 1
    return {'window_length': window_length, 'polyorder': polyorder}

#3.6 Filtering Methods
def Overall_Decrease (df_filtered, filename):
    """
    The function that accpets files if they are decreasing sets of data (polarization evolution is a decay ALWAYS)
    Will save plots of good files in PlotResults so that the user can see what has been accepted
    Will save plots of bad files in FailuresTest so that the user can see the polts being discarded.
    IMPORTANT: IF YOU ADD A NEW EXPERIMENT AND YOU ARE CONFIDENT IT IS A GOOD FILE BUT IS SENT TO FailuresTest PLEASE MANUALLY ADD THE TXT FILE TO THE force_accept_files. The program should rename eveything in SeparatedFolder so you can get the name from there  adding _Filtered if it is missing
    """
    FilteringMethodInt = 12 
    """
    This int chooses the method for filtering the wrong data. The options are:
        FilteringMethodInt = 0   
        FilteringMethodInt = 1
        FilteringMethodInt = 2
        FilteringMethodInt = 12
        FilteringMethodInt = 3
    """
    
    force_accept_files = [
        "PolarizationD3_EuAgAs_29_8_23_0_MillerIndex_(0,0,4)_Filtered.txt",
        "PolarizationD3_EuAgAs_1_30_8_23_5_MillerIndex_(3,0,0)_Filtered.txt",
        "PolarizationD3_SmI3_26_9_23_1_MillerIndex_(0,3,0)_Filtered.txt",
        "PolarizationD3_SmI3_1_28_9_23_3_MillerIndex_(0,3,0)_Filtered.txt",
        "PolarizationD3_MnSn-c_hor_1_22_3_24_1_MillerIndex_(3,0,0)_Filtered.txt",
        "PolarizationD3_MnSn-c_hor_15_3_24_0_MillerIndex_(0,0,2)_Filtered.txt",
        "PolarizationD3_MnSn-c_ver_14_3_24_1_(1,0,0)_Filtered.txt"]
    force_reject_files = [
        "PolarizationD3_EuAgAs_1_29_8_23_0_MillerIndex_(1,1,1)_Filtered.txt",
        "PolarizationD3_MnSn-c_ver_14_3_24_1_MillerIndex_(1,0,0)_Filtered.txt"] #The values were in the 0.3 to 0.2 range. Not a polarization
    
    """
    Some files failed the FilteringMethodInt = 12 method despite being considered good files. Others passed the test but were clearly tampered.
    These lists override the testing and accepts/rejets them immediately
    """


            
    if FilteringMethodInt == 12:
        """
        This method combine method 1 and 2 (that is the reason why it is called 12)
        It computes a score depending on how many derivatives are negative, (200 / (200 - percent_neg) - 1) to be precise. This is a normalized score (0-1) with a 1/x evolution
        It computes a score depending of the size of N, 2 * (-0.5 + 1 / (1 + needed_N / 8.54e-2)) to be precise. 8.54e-2 is the maximum of the data set. If a new maximum is achieved, the score wont be normalized (0-1) but won't break
        The final score combines both of these values (addition). Manually I have seen that 1.4 is a good threshold. If Score>1.4 the file will be accepted. If it is smaller it will be discarded by the main code (a False will be returned)
        Does the m<0 test
        Write eveythin in Summary_txt (filename, Score associated with N, Score associated with Derivatives nad the total Score). If the file was chosen to be forcefully accepted or rejected, a string will appear in the .txt file.
        Saves plots of both filters in PlotResults if it is correct and in FailureTest if it is considered a bad file. Again, if a new file is added it may be wise to check your experiment in these folders. For more info read the description of FilteringMethodInt = 0
        """
        output_folder = Path.cwd() / "CrystallinePlotResults"

        failures_folder = Path.cwd() / "CrystallineFailuresTest"
        failures_folder.mkdir(exist_ok=True)
    
        # Ensure paths are Windows long-path safe
        output_folder = Path(win_long_path(output_folder))
        failures_folder = Path(win_long_path(failures_folder))

  

        is_good = True


    
        def make_clean_name(filename: str) -> str:
            """
            Strip unnecessary prefixes/suffixes from filenames.
            Example:
            PolarizationD3_155K_2_18_9_23_1_MillerIndex_(4,0,1)_Filtered.txt
              -> 155K_2_18_9_23_1_(4,0,1)
            """
            base = Path(filename).stem  # remove extension
            if base.startswith("PolarizationD3_"):
                base = base[len("PolarizationD3_"):]
            if base.endswith("_Filtered"):
                base = base[:-len("_Filtered")]
            base = base.replace("MillerIndex_", "")
            return base

        # Build a clean name for titles / saving
        clean_name = (
            filename.replace("PolarizationD3_", "")
                    .replace("MillerIndex_", "")
                    .replace("_Filtered.txt", "")
        )

        def clean_plot_filename(filename: str, needed_N: Optional[float], plot_folder: Path) -> Path:
            """
            Build a clean filename for linear fit plots.
            Example:
            155K_2_18_9_23_1_(4,0,1)_N_4.30e-03.png
            or
            155K_2_18_9_23_1_(4,0,1)_NoNFound.png
            """
            base = make_clean_name(filename)
            suffix = f"_N_{needed_N:.2e}" if needed_N else "_NoNFound"
            return plot_folder / f"{base}{suffix}.png"
        
        def derivative_plot_filename(filename: str, plot_folder: Path) -> Path:
            """
            Build a clean filename for derivative plots.
            Example:
            CaFeAl_13_7_6_24_2_(0,0,2)_Derivatives.png
            """
            base = make_clean_name(filename)
            return plot_folder / f"{base}_Derivatives.png"
        x = df_filtered['DeltaTime'].values
        y = df_filtered['SoftPolarizationD3'].values
        Err = df_filtered['ErrPolarizationD3'].values if 'ErrPolarizationD3' in df_filtered.columns else np.zeros_like(y)

        
        def make_clean_name(filename: str) -> str:
            """
            Turn e.g.
              PolarizationD3_CaFeAl_13_7_6_24_2_MillerIndex_(0,0,2)_Filtered.txt
            into:
              CaFeAl_13_7_6_24_2_(0,0,2)
            and handle cases where filename contains '/' or '\\' (dates like DD/MM/YY).
            """
            s = str(filename).replace("/", "_").replace("\\", "_")  # prevent path splitting
            base = Path(s).stem  # remove extension if present
            if base.startswith("PolarizationD3_"):
                base = base[len("PolarizationD3_"):]
            if base.endswith("_Filtered"):
                base = base[:-len("_Filtered")]
            base = base.replace("MillerIndex_", "")
            return base
        
        def extended_area_plot_filename(filename: str) -> str:
            """EuAgAs_5_31_10_23_0_(3,0,0)_ExtendedArea.png"""
            return f"{make_clean_name(filename)}_ExtendedArea.png"

        return is_good



        
    
#3.7 Function that decides the correct set of Miller Indices    
def filter_best_combination(i,
    df,
    filter_func,
    filter_column_idx,
    new_column_name,
    filter_params_func,
    min_points_required=3,
    tolerance=1e-8,
    time_column_idx=None,
    error_column_idx=None):
    """
    Inputs:
        i is the integer that separates the array files (the original .fli files have more than one experiment with different headers. Each experiment is a different i.
        df is the pandas dataframe (the data)
        filter_func is the type of filter to smooth the data. Only savgol_filter can be used
        filter_column_idx is the name of the column in df that will be used for filtering ad all the other tests. Always choose 'PolarizationD3'
        new_column name is the name that will be added to the new column. If you cange it from SoftPolarizationD3 you might need to change manually this name in the rest of the code
        filter_params_func asks for the parameters for the filter. The function savgol_params_func was made especifically for this 
        tolerance is a measurement to know it the filter was changed anything or not
        time_column_idx works like filter_column_idx. Please keep it as 'DeltaTime'
        error_column_idx works like filter_column_idx. Please keep it as 'ErrPolarizationD3'
    Outputs:
        df_filtered is the df dataframe with the new column for SoftPolarization and only the data points of the Miller Inidices Combination that has passed all the filters
        PrettyCombination is the Miller Indices combination that has passed the filters. Should be something like (4,1,0) 
    Try each Miller index combination for a set i value, apply a filter, and return filtered df + PrettyCombination.
    Only adds the filtered column if enough points & data changes significantly. If it doesn't change too much, the column PolarizationD3 will be duplicated with the new name
    
    """
    from collections import defaultdict
    folder_name = FileName.replace(".fli", "")
    
    


    # Group by first three columns (Miller indices)
    combination_counts = (
        df.groupby([df.columns[0], df.columns[1], df.columns[2]])
        .size()
        .sort_values(ascending=False)
    )
    log_message(f"Analyzing combinations in file: {folder_name}_Array_{i}.fli")
    #Read the three numbers from the .fli file
    for comb, count in combination_counts.items(): 
        log_message(f"Combination {comb} occurs {count} times in file {folder_name}.fli. Trying this combination")
        mask = (
            (df.iloc[:,0] == comb[0]) &
            (df.iloc[:,1] == comb[1]) &
            (df.iloc[:,2] == comb[2])
        )
        PrettyCombination = format_combination(comb)
        

        filtered_df = df.loc[mask].copy()
        # Requisites for the Combination to be valid:
        # Requisite 1: Have data in the data
        if filtered_df.empty:
            log_message(f"      {PrettyCombination} has no data")
            continue
        
        # Requisite 2: Check if data column exists
        if filtered_df.shape[1] <= filter_column_idx:
            log_message(f"      Expected column index {filter_column_idx} not found. Skipping combination {PrettyCombination}")
            continue
        
        # Convert to numeric all columns (all columns are considered as object type)
        filtered_df = filtered_df.apply(pd.to_numeric, errors='coerce')
        filtered_df = filtered_df.dropna()  # drops any rows with NaNs introduced by coercion
        
        # Check dtypes
        all_numeric = all(dtype.kind in ('f', 'i') for dtype in filtered_df.dtypes)
        
        if all_numeric:
            log_message(f"      All columns have been successfully converted to numbers.")
        else:
            log_message(f"      Not all columns are numbers. Current dtypes:")
            log_message(f"      {filtered_df.dtypes}")
            log_message(f"      Expect Error Message from Python. Perhaps removing this file might be wise unless all files have the same issue")
        if filtered_df.empty:
            log_message(f"      All rows dropped after conversion to numeric. Skipping combination {PrettyCombination}")
            continue
        
        # Requisite 3: Polarization is ALWAYS positive. If any is negative, that is not a polarization. Immediately sent to the Bad Files Folder
        if (filtered_df.iloc[:, filter_column_idx] < 0).any():
            filename = f"PolarizationD3_{folder_name}_{DD}_{MM}_{YY}_{i}_MillerIndex_{PrettyCombination}_Filtered.txt"
            badfile_subfolder = BadFilesFolder / folder_name
            badfile_subfolder.mkdir(parents=True, exist_ok=True)
            badfiles_txt_path = badfile_subfolder / filename
            filtered_df.to_csv(win_long_path(badfiles_txt_path), index=False, sep='\t')
            log_message(f"      {PrettyCombination} has negative polarization values. Sent to BadFiles with name {filename}. Skipping to next Combination")
            continue


        
        # Requisite 4: Have at least three rows (otherwise we can't teach the ML algorithm anything).
        if len(filtered_df) < 2:
            filename = f"PolarizationD3_{folder_name}_{DD}_{MM}_{YY}_{i}_MillerIndex_{PrettyCombination}_Filtered.txt"
            badfile_subfolder = BadFilesFolder / folder_name
            badfile_subfolder.mkdir(parents=True, exist_ok=True)
            badfiles_txt_path = badfile_subfolder / filename
            filtered_df.to_csv(win_long_path(badfiles_txt_path), index=False, sep='\t')
            log_message(
                f"      {PrettyCombination} has only {len(filtered_df)} rows (< {min_points_required}). "
                f"Sent to BadFiles with name {filename}. Skipping to next Combination"
            )
            continue


        # Requisite 5: Be worthy of having the filter used.
        y = filtered_df.iloc[:, filter_column_idx].values
        filter_params = filter_params_func(len(y))
        try:
            y_filtered = filter_func(y, **filter_params)
            diff = np.abs(y - y_filtered)
            changed_count = np.sum(diff > tolerance)
            filtered_df[new_column_name] = y_filtered
            if changed_count > 0:
                log_message(f"      Filter changed {changed_count}/{len(y)} points. Adding column '{new_column_name}'.")
            else:
                log_message(f"      Filter applied but data unchanged. Adding '{new_column_name}' as duplicated values.")
            
            
        
        except Exception as e:
            log_message(f"      Error applying filter to combination {comb}: {e}")
            log_message(f"      Adding '{new_column_name}' as duplicated values to proceed anyway.")
            # Just duplicate the original column
            y_filtered = y.copy()
            filtered_df[new_column_name] = y_filtered
        #log_message(filtered_df)
        # Requisite 5: Add the Derivative and BandWidth filtering logic
        if Overall_Decrease(filtered_df, f"PolarizationD3_{folder_name}_{DD}_{MM}_{YY}_{i}_MillerIndex_{PrettyCombination}_Filtered.txt"):
            log_message(f"      {PrettyCombination} has surpassed all tests. Proceding with it.")
            return filtered_df, PrettyCombination
        else:
            log_message(f"      {PrettyCombination} failed the Overall_Decrease test. Trying next combination.")
            continue
    return None, None

#3.8 Function that removes the points considered bad and fixes the underestimated Uncertainty  
def RemoveOutcast_FixUncertainty(df_filtered, PrettyCombination, filename, AcceptableMultiplier=2.0, ShowPlot=False):
    """
    Repeats the process of obtaining the area in m*x+n-N < y < m*x+n+N where 75% of the points are inside the area.
    Multiplies the value of N by a factor AcceptableMultiplier and erases all points outisde this bigger area.
    As uncertainties are clearly underestimated I tried to make them reasonable (looking at the dispersion of the points it is clear there is systematic uncertainties
    Under the hypothesis that the polarization curve should be a soft curve (at least C^1) we will try to use χ^2 to add a provisional uncertainty margin fitting to a linear expression.
    This is a very inaccurate uncertainty increase but it is an improvement of the underestimated uncertainties (and the lack of ways to quantify the systematic uncertainty sources)
    The enlarged area will be plotted and saved in PlotResults for both the normal data and the softened data (Savitzky–Golay filter)
    """

    return df_filtered



"""
4. ZIP FOLDER TREATMENT
"""

"""
To make it easier for the user to use, you can just download the "processed" files in the ILL data base using the zip file. To avoid duplications
and other issues, the files won't be deleted. This reduces speed as it needs to reprocess all the files but we make sure that it works
This code and the next will take all the zip files, extract them, remove duplicates using the name AND hash sha256.
"""
to_erase = [
    "CrystallineLog_Predicting_Creation.txt",
    "CrystallinePolariserAndAnalyser_IDs.txt",
    "CrystallineSeparatedFolder",
    "CrystallinePlotResults",
    "CrystallineMLDataBase",
    "PolarizationTimeReference.txt",
    "CrystallineDataBase",
    "CrystallineBadFiles", 
    "CrystallineFailuresTest", 
    "CrystallineCell_ID.txt"
]
for item in to_erase:
    path = os.path.abspath(item)  # full path
    if os.path.exists(path):
        try:
            if os.path.isfile(path):
                os.remove(path)
                log_message(f"Deleted file: {path}")
            elif os.path.isdir(path):
                shutil.rmtree(path)
                log_message(f"Deleted folder: {path}")
        except Exception as e:
            log_message(f" Could not delete {path}: {e}")
    else:
        log_message(f"Not found (skipped): {path}")
# Path to CrystallinePredictFiles inside ML
predict_path = os.path.abspath(os.path.join("..", "ML", "CrystallinePredictFiles"))

if os.path.exists(predict_path):
    try:
        if os.path.isdir(predict_path):
            shutil.rmtree(predict_path)
            log_message(f"Deleted folder: {predict_path}")
        else:
            os.remove(predict_path)
            log_message(f"Deleted file (unexpected, was not a folder): {predict_path}")
    except Exception as e:
        log_message(f"Could not delete {predict_path}: {e}")
else:
    log_message(f"Not found (skipped): {predict_path}")        
def file_hash(filepath, algo="sha256", block_size=65536):
    """Compute hash of a file (default SHA256)."""
    h = hashlib.new(algo)
    with open(win_long_path(filepath), "rb") as f:
        for block in iter(lambda: f.read(block_size), b""):
            h.update(block)
    return h.hexdigest()

folder = Path("D3Files")  # Folder where all the raw data folders reside
zip_files = [f.name for f in folder.glob("*.zip")]  # List all zip files

log_message(f"Reading ZIP files. Checking for true duplicates by content...")
base_names = set()
seen_hashes = {}

for zip_file in zip_files:
    zip_path = folder / zip_file
    name = Path(zip_file).stem
    ext = Path(zip_file).suffix


    # Compute hash of this file
    filehash = file_hash(zip_path)

    if filehash in seen_hashes:
        log_message(f"Duplicate confirmed by hash! Removing: {zip_file} (same as {seen_hashes[filehash]})")
        #os.remove(win_long_path(zip_path))   # Safe long path
    else:
        seen_hashes[filehash] = zip_file
        base_names.add(name)

log_message(f"\n All duplicates (by content) removed. Begin unzipping...\n")

# Refresh zip_files list after removals
zip_files = [f.name for f in folder.glob("*.zip")]

# Unzip and remove original zip files
for zip_file in zip_files:
    zip_path = folder / zip_file
    if zipfile.is_zipfile(win_long_path(zip_path)):
        folder_name = sanitize(zip_file.stem if isinstance(zip_file, Path) else os.path.splitext(zip_file)[0])
        extract_dir = folder / folder_name
        log_message(f"Unzipping: {zip_file} -> {extract_dir}")
        try:
            with zipfile.ZipFile(win_long_path(zip_path), 'r') as zip_ref:
                zip_ref.extractall(win_long_path(extract_dir))
        except Exception as e:
            log_message(f"WARNING: Error extracting {zip_file}: {e}")
    else:
        log_message(f"WARNING: Skipping invalid zip file: {zip_file}")

log_message(f"\n Finished Unzipping. Experiments stored in individual folders substituting the zip files\n")

# --- Extraction of .fli files ---
source_folder = Path("D3Files")
# Destination folder at the same level as the notebook
database_folder = (Path.cwd() / "CrystallineDataBase").resolve()

# Ensure parent exists
database_folder.parent.mkdir(parents=True, exist_ok=True)

# Now create the folder
os.makedirs(win_long_path(database_folder), exist_ok=True)

log_message(f"\n\n\n Scanning all folders for .fli files...\n ")
for item in os.listdir(win_long_path(source_folder)):
    item_path = source_folder / item  # Path object
    if item_path.is_dir():
        log_message(f"Processing folder: {item_path.name}")
        for root, dirs, files in os.walk(win_long_path(item_path)):
            for file in files:
                if file.lower().endswith(".fli"):
                    src_file = Path(root) / file
                    dest_file = database_folder / file

                    # Handle duplicate names
                    counter = 1
                    base_name = Path(file).stem
                    ext = Path(file).suffix
                    while (database_folder / f"{base_name}_{counter}{ext}").exists():
                        counter += 1
                    dest_file = database_folder / f"{base_name}_{counter}{ext}"

                    log_message(f"Copying: {src_file} -> {dest_file}")
                    shutil.copy2(win_long_path(src_file), win_long_path(dest_file))

        log_message(f"Deleting folder: {item_path}")
        shutil.rmtree(win_long_path(item_path))


log_message(f"\n All .fli files collected, sent from folder {source_folder} to folder {database_folder}.\n")

"""
6 SEPARATION OF FLI FILES ACCORDING TO EXPERIMENTS
"""

"""
Some fli files have the wrong structure (they are not polarization measurementes) and if they are polarization files they can have more than one experiment per file.
For evey fli file we will read the contents and try to find the header (a string in an entire line). This symbolizes the beginning of an experiment
If there are numerical values before the first header, that means that the process of saving the file occured before changing something of the experiment. These data rows will be skipped
A correct fli file will have the following structure:
    polariser cell info ge18004 pressure/init. polar 2.29 0.79 initial date/time 17 09 23 @ 10:39
    37391   4.000   0.000   1.000 18/09/23 06:20:44     155.03  +z +z     0.8391    0.0156   11.4270    1.2031     120.00
    37392   4.000   0.000   1.000 18/09/23 06:26:49     155.05  +x +x     0.8255    0.0110   10.4610    0.7211     300.00
    ...

Which corresponds to the following information:
    String:'polariser cell info', CellID, String:'pressure/init. polar', Pressure(unknown units), InitialLabPolarization, String:'date/time', Day, Month, Year, String:'@', Hour:Minute
    Measurement Number, First Miller Index, Second Miller Index, Third Miller Index, Date Of Measurement, Time Of Measurement, Temperature [Kelvin],
                        Direction Of Polarization In The First Polarizer Cell (Direction of the quantum operator S_x,S_y,S_z), Direction Of Polarization In The Second Polarizer Cell,
                        Polarization, Polarization uncertainty, Flipping Ratio, FlippingRatio Uncertainty, Duration of the measurement

The direction +z is chosen to be pointing away from the ground.
The direction +x is the direction of the flow of neutrons, i.e, the direction of Scattering.
The direction +y is the orthogonal to both of them.
D3 uses two polariser cells, one between the reactor and the sample and a second between the sample and the sensor. The first one guarantees that only neutrons with the correct spin direction
interacts with the sample. The second one guarantees that only the neutrons that have unchanged spin direction after interacting with the sample are detected by the sensor. This is
the reason why the directions (+z,+y,+x,-z,-y,-x) appear twice.
We have considered that temperature is not a relevant factor and the flipping ratio has no new information that polarization alrady posseses.
First, the code will first locate the first header (ignoring eveything before) and save all the data afterwards (until the next header or end of the document) in a file with the suffix Array_{i} (i is the number of headers already processed in that fli file)
Second, it will save the header as a file with the suffix Parameters.
Third, the header row and the columns of Measurement Number, Temperature, Flipping Ratio, FlippingRatio Uncertainty and Time Between Measurements will be erased
Fourth, as all data measurement uses the +z,+z combination, all other combinations are erased
Fifth, not all data from all Miller Index combinations are polarization measurements. Even some of the ones that are polarization measurements are tampered (playing with magnetic fields for example).
This means that there needs to be a way to select the correct combination. For starters, irrational Miller indices are not used for measurements with the samples (they need to be discarded)
The integer Miller indices combination will be put to the test by all the functions defined before.
For evey succesful experiment we will output:
    Image:  "PolarizationD3_{folder_name}_{DD}/{MM}/{YY}_{i}_MillerIndex_{PrettyCombination}_Multiplier={Multiplier}.png" in PlotResults. Shows the plot with the extended area with the raw data
    Image:  "PolarizationD3_{folder_name}_{DD}/{MM}/{YY}_{i}_MillerIndex_{PrettyCombination}_Multiplier={Multiplier}_Soft.png" in PlotResults. Shows the plot with the extended area with the filtered data
    Image:  "PolarizationD3_{folder_name}_{DD}/{MM}/{YY}_{i}_MillerIndex_{PrettyCombination}_Filtered.txt_plot_Derivatives.png" in PlotResults. Shows the evolution of the "derivatives"
    Image:  "PolarizationD3_{folder_name}_{DD}/{MM}/{YY}_{i}_MillerIndex_{PrettyCombination}_Filtered.txt_N_{N}_ManualInterval.png" in PlotResults. Shows the plot with the non-exteded area
    Txt:    "PolarizationD3_{folder_name}_{DD}/{MM}/{YY}_{i}_MillerIndex_{PrettyCombination}.txt" in MLDataBase. It contains the four data columns (DeltaTime, PolarizationD3, SoftPolarizationD3, ErrPolarizationD3)
    Txt:    "PolarizationD3_{folder_name}_{DD}/{MM}/{YY}_{i}_MillerIndex_{PrettyCombination}_Parameters.txt" in MLDataBase. It contains the parameters (CellID, Pressure, LabPolarization, LabTime)
These plots are not necessary but are saved for the user to know what all the files look like.
The files that are wrong or useless when all is done are the folowing:
    Txt:    "{folder_name}_Arrays_{i}.txt" in SeparatedFolder/{folder_name}. It still has the header and useless columns. It is the fli file of evey chunk, of every recorded experiment (correct or incorrect)
    Txt:    "PolarizationD3_{folder_name}_{DD}/{MM}/{YY}_{i}_MillerIndex_{PrettyCombination}.txt" in SeparatedFolder/{folder_name}. It is the same as the one in MLDataBase (a duplicate)
    Image:  "PolarizationD3_{folder_name}_{DD}/{MM}/{YY}_{i}_MillerIndex_{PrettyCombination}.png" in SeparatedFolder/{folder_name}. It plots (with error bars) PolarizationD3
    Image:  "PolarizationD3_{folder_name}_{DD}/{MM}/{YY}_{i}_MillerIndex_{PrettyCombination}_Combined.png" in SeparatedFolder/{folder_name}. It plots (with error bars) both PolarizationD3 and SoftPolarizationD3
    Image:  "PolarizationD3_{folder_name}_{DD}/{MM}/{YY}_{i}_MillerIndex_{PrettyCombination}_Softened.png" in SeparatedFolder/{folder_name}. It plots (with error bars) SoftPolarizationD3
    Folder: "FailuresTest" contains all the graphs of the data sets that were considered not worthy but had more points that the ones saved. Check them if your experiment was not properly added
    Folder: "DataBase" has the raw fli files. Once the code has been used they are no longer important (if you don't find the folder I may have added a line of code to erase it. Sorry in advance for any inconveniences)  
"""
# Path to the original folder and the final folder
DataBase = Path('CrystallineDataBase')
output_base = Path('CrystallineSeparatedFolder')

# List all .fli files in that folder, prepare folders
FileNameList = [f.name for f in DataBase.glob('*.fli')]
polyorder = 2
default_window_length = 5
SeparatedFolder = Path("CrystallineSeparatedFolder")
BadFilesFolder = Path("CrystallineBadFiles")
MLDataBaseFolder = Path("CrystallineMLDataBase")
BadFilesFolder.mkdir(exist_ok=True, parents=True)
MLDataBaseFolder.mkdir(exist_ok=True, parents=True)
log_message(f"\n\n Files in the data base that will be (tried) to be used\n {FileNameList}\n")

for FileName in FileNameList:
    """READ THE FILE AND SEPRATE IT INTO EACH EXPERIMENT USING THE POLARIZATION CELL"""
    # 1- Open file
    folder_name = FileName.replace(".fli", "")
    output_folder = output_base / folder_name
    file_path = DataBase / FileName
    output_folder.mkdir(parents=True, exist_ok=True)

    with open(win_long_path(file_path), "r", encoding='utf-8', errors='ignore') as f:
        lines = f.readlines()

    
    # 2- Locate the header with CellID, Pressure, etc. Chunks are the data rows sandwiched between two 'polariser cell info' strings
    chunks = []
    current_chunk = []
    started = False  # flag to know when we found first header

    for line in lines:
        if line.strip().startswith("polariser cell info"):  # All before polariser cell info will be forgotten
            if started and current_chunk:
                chunks.append(current_chunk)
            current_chunk = [line]
            started = True
        else:
            if started:
                current_chunk.append(line)
            # else: we are before the first header, so ignore these lines

    if not started:
        log_message(f" File '{FileName}' does NOT contain any 'polariser cell info' header. Skipping.\n")
        continue
    else:
        log_message(f" File '{FileName}' contains at least one 'polariser cell info' header.")

    if current_chunk:
        chunks.append(current_chunk)

    #3- Save .fli files for every correct chunk
    base_name = FileName.replace(".fli", "")  # remove .fli for clean filenames
    log_message(f"Creating all the Array files \n")
    for i, chunk in enumerate(chunks):
        log_message(chunk)
        fli_filename = f"{base_name}_Arrays_{i}.fli"
        fli_path = output_folder / fli_filename
        with open(fli_path, "w") as f_out:
            f_out.writelines(chunk)  # each row is written exactly as it was

    # 4- As CellID can be exchanged with real parameters, it is written in an independent file
    cell_id_file = Path.cwd() / "Crystalline_CellID.txt"
    try:
        with open(win_long_path(cell_id_file), 'r', encoding='utf-8') as file:
            seen_strings = set(line.strip() for line in file)
    except FileNotFoundError:
        seen_strings = set()
    
    # 5- Open each Array file and work with it (The Array file still has the header)
    with open(win_long_path(cell_id_file), 'a', encoding='utf-8') as file:
        for i in range(len(chunks)):
            FLI_filename = f"{base_name}_Arrays_{i}.fli"  # Name of the Array file
            FLI_path = output_folder / FLI_filename  # Full path
            if not FLI_path.exists():
                log_message(f"WARNING: Array file does not exist: {FLI_path}")
                continue
            df = pd.read_csv(win_long_path(FLI_path), sep=r'\s+', header=None, on_bad_lines='skip')  # Read file
            log_message(f"Reading {FLI_path}, removing ***WARNING No centering scan found ")
            warning_str = "***WARNING No centering scan found"

            #5.1 Combine first 4 columns as strings, join them with space, and filter rows containing this phrase (it is not important for us)
            df = df[~df.iloc[:, :5].astype(str).agg(' '.join, axis=1).str.contains('No centering scan found', regex=False)] 
            
            #5.2 Extract useful information from the header. Hopefully, CellID, Pressure, LabPolarization, Year, Month, Day, time of lab measurement before first experiment measurement (negative time) will be stored locally
            log_message(f"Header Information Extraction...")
            CellID =          df.iloc[0].tolist()[3]
            Pressure =        df.iloc[0].tolist()[6]
            LabPolarization = df.iloc[0].tolist()[7]

            try:
                HM, DD, MM, YY = df.iloc[0].tolist()[14], int(df.iloc[0].tolist()[10]), int(df.iloc[0].tolist()[11]), int(df.iloc[0].tolist()[12])
                Day_Ref = f"{DD:02d}/{MM:02d}/{YY:02d}"
                dt = Time(Day_Ref, HM)
            except Exception as e:
                log_message(f"Skipping file {file_path} because of invalid header data: {e}")
                continue

            
            #5.3 All redundant/useless information is removed
            log_message(f"Removing Measurement Index, Temperature, Flipping Ratio, Uncertainty of Flipping Ratio and Time between measurements,...")
            df = df.iloc[1:].reset_index(drop=True)
            df = df.drop(df.columns[0], axis=1)
            df = df.drop(df.columns[5], axis=1)
            df = df.drop(df.columns[9], axis=1)
            df = df.drop(df.columns[9], axis=1)
            df = df.drop(df.columns[9], axis=1)
            df = df.drop(df.columns[9], axis=1)
            log_message(f"Saving only polarization values for the Spin Directions wanted in both Polarizer Cells, i.e. (+z,+z)")

            #5.4 Keep only rows where both are +z
            df = df[(df[7] == '+z') & (df[8] == '+z')].copy()
            if df.empty:
                log_message(f"No valid '+z' rows in file {FileName}_Arrays_{i}.fli, skipping")
                continue  # skip to next file in your loop
            df = df.drop(df.columns[[5,6]], axis=1, errors='ignore')
            #5.5 Convert Miller index columns into integers. From string or object to float and if the float is close to an integer (tolerance is 1e-8) then save as integer. Otherwise remove row
            cols_to_convert = [1, 2, 3]
            df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce').astype(float)            
            mask = np.isclose(df[cols_to_convert], np.round(df[cols_to_convert]), atol=1e-8)
            df = df[mask.all(axis=1)].copy()
            log_message(f"All Spin directions removed. All irrational Miller Indices removed. Adding DeltaTime")
            
            #5.5 The time columns are converted into difference of time being the referenced time the first +z,+z measurement that has survived at this point
            if df.shape[0] < 2:
                log_message(f"Not enough valid rows after filtering, skipping chunk")
                continue
            df['DeltaTime'] = df.apply(
                lambda row: deltatime(df[4].iloc[0], df[5].iloc[0], row[4], row[5]), axis=1 )
            ref_dt = Time(df[4].iloc[0], df[5].iloc[0])
            LabTime = int((dt - ref_dt).total_seconds())
            
            #5.5 The time columns are converted into difference of time being the referenced time the first +z,+z measurement that has survived at this point
            df['DeltaTime'] = df.apply(
                lambda row: deltatime(df[4].iloc[0], df[5].iloc[0], row[4], row[5]), axis=1 )
            ref_dt = Time(df[4].iloc[0], df[5].iloc[0])
            with open("PolarizationTimeReference.txt", "a") as f:
                f.write(str(ref_dt) + "\n")
            LabTime = int((dt - ref_dt).total_seconds())

            #5.6 Rename the columns PolarizationD3, ErrPolarizationD3 (the polarization column and its uncertainty). The other one with name is DeltaTime. The rest are numbers (will be erased).
            #Also we remove the time strings (with DeltaTime they have no new information)
            log_message(f"Renaming PolarizationD3 and ErrPolarizationD3")
            df.rename(columns={
                df.columns[5]: 'PolarizationD3',
                df.columns[6]: 'ErrPolarizationD3'
            }, inplace=True)
            df.drop(columns=[df.columns[3], df.columns[4]], inplace=True)
            log_message(f"Dropped Time Strings")

            
            #5.7 Begin filtering and softening with previous functions
            log_message(f"Begin removal of Bad files and softening with Savitzky-Golay filter")
            filtered_df, PrettyCombination = filter_best_combination(i,
                df,
                filter_func=savgol_filter,
                filter_column_idx=df.columns.get_loc('PolarizationD3'),
                new_column_name='SoftPolarizationD3',
                filter_params_func=savgol_params_func,
                min_points_required=3,
                tolerance=1e-8,
                time_column_idx=df.columns.get_loc('DeltaTime'),
                error_column_idx=df.columns.get_loc('ErrPolarizationD3')
                )
            #If nothing survived the filters/purge then use'continue' and go for the next experiment
            if filtered_df is None and PrettyCombination is None:
                log_message(f"Chunk {i}: No suitable combination found. Skipping to next chunk or file.")
                log_message(f"_______________________________________________________________\n")
                continue  # skip to next chunk
            
            #5.8 Removal of Miller indices (we have all the information they could give us)
            log_message(f"Removing Miller Indices columns")
            #log_message(filtered_df)
            filtered_df = filtered_df.iloc[:, 3:]
            desired_order = ["DeltaTime", "PolarizationD3", "SoftPolarizationD3", "ErrPolarizationD3"]

            
            # 5.9 Remove the points that won't be useful for the ML algorithm
            columns_to_save = [col for col in desired_order if col in filtered_df.columns]  # Keep only the columns that exist
            df_SEMIFINAL = filtered_df[columns_to_save].copy()
            df_FINAL = filtered_df = df_SEMIFINAL
            
            

            # 5.11 Save the files
            log_message(f"Finally we save the chunk")
            csv_filename = f"PolarizationD3_{folder_name}_{DD}_{MM}_{YY}_{i}_MillerIndex_{PrettyCombination}.txt"
            Parameter_filename = f"PolarizationD3_{folder_name}_{DD}_{MM}_{YY}_{i}_MillerIndex_{PrettyCombination}_Parameters.txt"
            csv_path = output_folder / csv_filename
            csv_path.parent.mkdir(parents=True, exist_ok=True)

                
            # Save CSV

            df_FINAL.to_csv(win_long_path(csv_path), index=False, sep=',')
            
            # Save a copy to ML database
            ml_txt_path = MLDataBaseFolder / csv_filename
            ml_txt_path.parent.mkdir(parents=True, exist_ok=True)

            df_FINAL.to_csv(win_long_path(ml_txt_path), index=False, sep=',')
            log_message(f"Saved: {csv_filename}")
            
            # Parameter file
            log_message(f"Processing Parameter file...")
            ml_param_path = MLDataBaseFolder / Parameter_filename
            ml_txt_path.parent.mkdir(parents=True, exist_ok=True)

            with open(win_long_path(ml_param_path), 'w', encoding='utf-8') as f:
                f.write("CellID,Pressure,LabPolarization,LabTime\n")
                f.write(f"{CellID},{Pressure},{LabPolarization},{LabTime}")
            log_message(f"Saved: {Parameter_filename}")
            log_message(f"Parameter and Array files saved to ML database: {MLDataBaseFolder}\n_______________________________________________________________\n\n")
        else:
            log_message(f"{FileName}_Arrays_{i}.fli is empty, skipping this file.\n")
            continue  

    #5.12 Remove unwanted folders and files
    for i in range(len(chunks)):
        temp_filename = f"{base_name}_Arrays_{i}.fli"
        temp_path = output_folder / temp_filename
        try:
            temp_path.unlink()
        except FileNotFoundError:
            pass        
    log_message(f"Created and saved {len(chunks)} CSV files from file called {FileName}.")
    if output_folder.exists() and not any(output_folder.iterdir()):
        output_folder.rmdir()
        log_message(f"Removed empty folder: {output_folder}")
    log_message('\n\n')

import glob
         


"""
7. CELLID FILE PROCESSING
"""
ml_database_folder = Path("CrystallineMLDataBase")

# Find all txt files whose names end with Parameters.txt (case insensitive)
parameter_files = list(ml_database_folder.glob('*Parameters.txt'))

log_message(f"Found {len(parameter_files)} parameter files.")

unique_cell_ids = []
seen = set()

for filepath in parameter_files:
    try:
        with open(win_long_path(filepath), 'r', encoding='utf-8') as f:
            lines = f.readlines()
            if len(lines) >= 2:
                second_row = lines[1].strip()
                parts = second_row.split(',')
                if parts:
                    cell_id = parts[0]
                    if cell_id not in seen:
                        seen.add(cell_id)
                        unique_cell_ids.append(cell_id)
    except Exception as e:
        log_message(f"Failed to read {filepath}: {e}")

# Write to Crystalline_CellID.txt
cellid_file = Path.cwd() / "Crystalline_CellID.txt"
with open(win_long_path(cellid_file), "w", encoding='utf-8') as f:
    for cell_id in unique_cell_ids:
        f.write(f"{cell_id}\n")

log_message(f"Saved {len(unique_cell_ids)} unique cell IDs to {cellid_file.name}.")

# Remove the separated folder
folder_to_delete = Path.cwd() / "CrystallineSeparatedFolder"
if folder_to_delete.exists():
    shutil.rmtree(win_long_path(folder_to_delete))
    log_message(f"Folder '{folder_to_delete}' has been deleted.")
else:
    log_message(f"Folder '{folder_to_delete}' does not exist.")

"""
Removal of Duplicates
"""
hash_map = defaultdict(list)

def file_sha256(filepath, block_size=65536):
    """Compute SHA256 hash of a file (safe for large files)."""
    sha256 = hashlib.sha256()
    with open(win_long_path(filepath), "rb") as f:
        while chunk := f.read(block_size):
            sha256.update(chunk)
    return sha256.hexdigest()

# Scan all .txt files (only base files without '_Parameters')
for root, _, files in os.walk(win_long_path(ml_database_folder)):
    for file in files:
        if file.lower().endswith(".txt") and "_parameters" not in file.lower():
            path = Path(root) / file
            file_hash = file_sha256(path)
            hash_map[file_hash].append(path)

# Report & delete duplicates
duplicates_found = False
for file_hash, paths in hash_map.items():
    if len(paths) > 1:
        duplicates_found = True
        log_message(f"\nDuplicate group (hash={file_hash}):")
        log_message(f"   Keeping: {paths[0]}")

        # All but the first are duplicates
        for p in paths[1:]:
            base_name, ext = os.path.splitext(p)
            param_file = Path(f"{base_name}_Parameters{ext}")

            try:
                os.remove(win_long_path(p))
                log_message(f"   Deleted duplicate base file: {p}")
            except Exception as e:
                log_message(f"   Could not delete base file {p}: {e}")

            # Also try deleting the corresponding parameter file
            if param_file.exists():
                try:
                    os.remove(win_long_path(param_file))
                    log_message(f"   Deleted parameter file: {param_file}")
                except Exception as e:
                    log_message(f"   Could not delete parameter file {param_file}: {e}")

if not duplicates_found:
    log_message("No duplicates found in MLDataBase!")
else:
    log_message("\n Duplicate cleanup complete!")
    
    
# Define paths relative to the notebook location
current_dir = Path().resolve()  # directory of FileLecture.ipynb
ml_database = current_dir / "CrystallineMLDataBase"
predict_files = current_dir.parent / "ML" / "CrystallineToPredictFiles"

# Make sure the destination exists
predict_files.mkdir(parents=True, exist_ok=True)

# --- Step 1: Clear the destination folder ---
for item in predict_files.iterdir():
    if item.is_file():
        os.remove(item)
        log_message(f"Deleted existing file: {item}")
    elif item.is_dir():
        shutil.rmtree(item)
        log_message(f"Deleted existing folder: {item}")

# --- Step 2: Move all files from CrystallineMLDataBase -> CrystallineToPredictFiles ---
for item in ml_database.iterdir():
    dest = predict_files / item.name
    shutil.move(win_long_path(item), win_long_path(dest))
    log_message(f"Moved {item.name} -> {predict_files}")

# --- Step 3: Remove the now-empty CrystallineMLDataBase ---
try:
    ml_database.rmdir()
    log_message(f"Removed empty folder: {ml_database}")
except OSError:
    log_message(f"Could not remove {ml_database}, not empty.")